In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from data_sources.clob import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

In [ ]:
# Constants
CONNECTOR_NAME = "binance"
INTERVALS = ["15m"]
DAYS = 7
FETCH_CANDLES = True

# Features configuration
VOLATILITY_WINDOW = 100
VOLUME_SHORT_WINDOW = 20
VOLUME_LONG_WINDOW = 100

# Download data
- Get trading rules
- Get candles for the last x days

In [ ]:
import asyncio

trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset("FDUSD")\
    .filter_by_min_notional_size(Decimal("1"))\
    .get_all_trading_pairs()

In [ ]:
BATCH_CANDLES_REQUEST = 100
SLEEP_REQUEST = 2.0


if FETCH_CANDLES:
    number_of_calls = (len(trading_pairs) // BATCH_CANDLES_REQUEST) + 1

    all_candles = {}

    for i in range(number_of_calls):
        print(f"Batch {i + 1}/{number_of_calls}")
        start = i * BATCH_CANDLES_REQUEST
        end = (i + 1) * BATCH_CANDLES_REQUEST
        print(f"Start: {start}, End: {end}")
        end = min(end, len(trading_pairs))
        trading_pairs_batch = trading_pairs[start:end]

        tasks = [clob.get_candles_last_days(
        connector_name=CONNECTOR_NAME,
        trading_pair=trading_pair,
        interval=interval,
        days=DAYS,
        ) for trading_pair in trading_pairs_batch for interval in INTERVALS]

        candles = await asyncio.gather(*tasks)
        candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}
        all_candles.update(candles)
        if i != number_of_calls - 1:
            print(f"Sleeping for {SLEEP_REQUEST} seconds")
            await asyncio.sleep(SLEEP_REQUEST)
    clob.dump_candles_cache(os.path.join(root_path, "data"))
else:
    clob.load_candles_cache(os.path.join(root_path, "data"))

In [ ]:
candles = [value for key, value in clob.candles_cache.items() if key[2] in INTERVALS and key[0] == CONNECTOR_NAME]

In [ ]:
from features.candles.volatility import VolatilityConfig
from features.candles.volume import VolumeConfig
from research_notebooks.dneitor.utils import generate_report

report = generate_report(
    candles=candles,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_SHORT_WINDOW, long_window=VOLUME_LONG_WINDOW))
report

In [ ]:
from research_notebooks.dneitor.utils import filter_top_markets

TOP_X_MARKETS = 10  # Number of top markets to select
MIN_VOLUME_USD = 2000  # Minimum volume in USD
MIN_NATR = 0.001  # Minimum ATR
TREND_THRESHOLD = -2  # Trend threshold

top_markets = filter_top_markets(report_df=report, top_x=TOP_X_MARKETS, min_volume_usd=MIN_VOLUME_USD, min_natr=MIN_NATR, trend_threshold=TREND_THRESHOLD)
top_markets_candles = {candle.trading_pair: candle for candle in candles if candle.trading_pair in top_markets["trading_pair"].values}
top_markets

In [ ]:

from research_notebooks.dneitor.utils import generate_config, dump_dict_to_yaml


TOTAL_AMOUNT = 3000  # General total amount for all markets
MIN_AMOUNT_PER_MARKET = 200  # Minimum amount per market
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_OPEN_ORDERS = 1  # Input max open orders for each market

TAKE_PROFIT_MULTIPLIER = 0.3  # Multiplier for take profit based on NATR
AMOUNTS_QUOTE_PCT = [0.1, 0.1, 0.1, 0.1, 0.2, 0.2]  # Weights for each cluster

strategy_config = generate_config(
    id=f"dneitor-{CONNECTOR_NAME}_0.4",
    connector_name=CONNECTOR_NAME,
    candles=top_markets_candles,
    top_markets=top_markets,
    total_amount=TOTAL_AMOUNT,
    amounts_quote_pct=AMOUNTS_QUOTE_PCT,
    activation_bounds=ACTIVATION_BOUNDS,
    take_profit_multiplier=TAKE_PROFIT_MULTIPLIER,
    max_open_orders=MAX_OPEN_ORDERS,
    min_amount_per_market=MIN_AMOUNT_PER_MARKET,
)

dump_dict_to_yaml("configs/", strategy_config)